<a href="https://colab.research.google.com/github/FajarKKP/Bottle_cap_color_classifier/blob/main/Bottle_cap_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This will be the jupyter notebook where the bottle cap classifier is used

Dataset prep.

So I have to re-label the datset with the following label:

0 = other
1 = light blue
2 = dark blue

I relabel it in roboflow due to its ease of use and the complimentary tools it have for data preparation.

When using roboflow, there is the option to prep the dataset (from preprocessing step until augmentation). Usually this is done when we are training on the model, not before. There are pros and cons on preprocess + augment dataset before or on the fly. The main pros is usually resource related, because we basically skip that step during training and has been prep beforehand. The cons is that the dataset will inflexible, if it does not fit what we are trying to achieve, the process is useless and we have to prep it again on the fly.

For this project, I only do the preprocessing step and figured to choose data augmentation during the model training. This is due to roboflow free account only letting me augment data into a limited images, which can effect badly on the final outcome.

For the preprocessing, I apply the Auto-orient step. It is used to re-orient images that may be saved in EXIF metadata. This is usefull to prevent bounding box misalignment due to EXIF metadata save.




Based on initial search on the web, YOLOv5-nano is the best starting model for this case. It has been proven in real-case issue. It is light compared to other model with same function with a good performance on edge device. It is a good starting point and benchmark.

In [1]:
# Set up and install yolov5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17731, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 17731 (delta 54), reused 31 (delta 31), pack-reused 17640 (from 3)
Receiving objects: 100% (17731/17731), 17.15 MiB | 24.53 MiB/s, done.
Resolving deltas: 100% (12044/12044), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
